# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import os
import json
import time

# Import API key
from api_keys import weather_api_key
from api_keys import geoapify_api_key
from api_keys import gmap_api_key


In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("/Users/vrindapatel/Documents/GitHub/python-api-challenge/output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Ushuaia,100,AR,1713758915,96,-54.8000,-68.3000,33.98,4.68
1,1,Edinburgh of the Seven Seas,45,SH,1713758915,89,-37.0676,-12.3116,63.75,22.41
2,2,Xiuyan,74,CN,1713758915,55,40.2928,123.2744,67.03,11.41
3,3,Ribeira Grande,100,PT,1713758916,76,38.5167,-28.7000,58.75,9.33
4,4,Sim,100,RU,1713758917,64,54.9930,57.6982,66.36,3.58


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
# create a map that display a marker for each city in the dataset of city_data_df
import gmaps
import gmaps.datasets
gmaps.configure(api_key=gmap_api_key
)

# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]
# Fill NaN values and convert to float
rating = city_data_df["Max Temp"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=[max(0, rating) for rating in rating], 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null value
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Filter the dataset to find the cities that fit the criteria
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & 
                                    (city_data_df["Max Temp"] < 80) & 
                                    (city_data_df["Wind Speed"] < 10) & 
                                    (city_data_df["Cloudiness"] == 0)].dropna()
ideal_weather_df





,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
11,11,Acapulco de Juárez,0,MX,1713758920,69,16.8634,-99.8901,73.06,2.06
54,54,Codrington,0,AU,1713758943,31,-38.2667,141.9667,70.39,8.84
176,176,Idfu,0,EG,1713758414,21,24.9803,32.8747,75.22,5.66
195,195,San Luis de la Loma,0,MX,1713758432,69,17.2706,-100.8939,76.51,1.07
255,255,Coahuayana de Hidalgo,0,MX,1713758489,88,18.7000,-103.6583,70.03,2.01
276,276,Nuriootpa,0,AU,1713758509,15,-34.4833,139.0000,77.02,8.48
295,295,Korla,0,CN,1713758527,19,41.7597,86.1469,73.15,5.03
425,425,São João da Barra,0,BR,1713758647,76,-21.6403,-41.0511,71.44,5.86
448,448,Bull Savanna,0,JM,1713758668,83,17.8868,-77.5902,74.57,7.02
455,455,Taoudenni,0,ML,1713758674,15,22.6783,-3.9836,77.14,8.50


In [47]:
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
11,11,Acapulco de Juárez,0,MX,1713758920,69,16.8634,-99.8901,73.06,2.06
54,54,Codrington,0,AU,1713758943,31,-38.2667,141.9667,70.39,8.84
176,176,Idfu,0,EG,1713758414,21,24.9803,32.8747,75.22,5.66
195,195,San Luis de la Loma,0,MX,1713758432,69,17.2706,-100.8939,76.51,1.07
255,255,Coahuayana de Hidalgo,0,MX,1713758489,88,18.7000,-103.6583,70.03,2.01


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add a "Hotel Name" column to the DataFrame.
# Display the result
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
11,Acapulco de Juárez,MX,16.8634,-99.8901,69,
54,Codrington,AU,-38.2667,141.9667,31,
176,Idfu,EG,24.9803,32.8747,21,
195,San Luis de la Loma,MX,17.2706,-100.8939,69,
255,Coahuayana de Hidalgo,MX,18.7000,-103.6583,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for a hotel
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 1000,
          "key" : gmap_api_key}

radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

Starting hotel search


In [50]:
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    print("------------")

# Print a message to follow up the hotel search
print("Hotel search completed")

# Display the hotel_df DataFrame
hotel_df


Retrieving Results for Index 11: Acapulco de Juárez.
Closest hotel in Acapulco de Juárez is Ramada by Wyndham Acapulco Hotel & Suites.
------------
Retrieving Results for Index 54: Codrington.
Missing field/result... skipping.
------------
Retrieving Results for Index 176: Idfu.
Closest hotel in Idfu is Mekkah Hotel.
------------
Retrieving Results for Index 195: San Luis de la Loma.
Closest hotel in San Luis de la Loma is HOTEL RUIZ.
------------
Retrieving Results for Index 255: Coahuayana de Hidalgo.
Closest hotel in Coahuayana de Hidalgo is Hotel Los Arcos.
------------
Retrieving Results for Index 276: Nuriootpa.
Closest hotel in Nuriootpa is Vine Inn Barossa.
------------
Retrieving Results for Index 295: Korla.
Closest hotel in Korla is The Garden Hotel Korla.
------------
Retrieving Results for Index 425: São João da Barra.
Closest hotel in São João da Barra is Pousada Carioca.
------------
Retrieving Results for Index 448: Bull Savanna.
Closest hotel in Bull Savanna is Big Red

,City,Country,Lat,Lng,Humidity,Hotel Name
11,Acapulco de Juárez,MX,16.8634,-99.8901,69,Ramada by Wyndham Acapulco Hotel & Suites
54,Codrington,AU,-38.2667,141.9667,31,
176,Idfu,EG,24.9803,32.8747,21,Mekkah Hotel
195,San Luis de la Loma,MX,17.2706,-100.8939,69,HOTEL RUIZ
255,Coahuayana de Hidalgo,MX,18.7000,-103.6583,88,Hotel Los Arcos
276,Nuriootpa,AU,-34.4833,139.0000,15,Vine Inn Barossa
295,Korla,CN,41.7597,86.1469,19,The Garden Hotel Korla
425,São João da Barra,BR,-21.6403,-41.0511,76,Pousada Carioca
448,Bull Savanna,JM,17.8868,-77.5902,83,Big Red's
455,Taoudenni,ML,22.6783,-3.9836,15,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
# drop the rows with missing hotel name
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df

# Add a hotel name and country as additional information in the hover message for each city in the map
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [70]:
import collections
collections.Iterable = collections.abc.Iterable

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))